siqa solo# Case Studies:
- VGG-16 (from Keras models repository, fine tuned)
- ResNet152
- XCeption

> https://www.topbots.com/important-cnn-architectures/

In [ ]:
# https://keras.io/guides/transfer_learning/

# Xception
Before training and fine tuning lets see the performance of the pretrained model on some samples.

## Data loading

In [ ]:
import numpy
import numpy as np
from torch.utils.data import DataLoader
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((299, 299), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test, batch_size=16, shuffle=True)

In [ ]:
import keras

label_mappings = {0: "chihuahua", 1: "muffin"}
untouched_xception = keras.applications.Xception(weights='imagenet')

In [ ]:
def evaluate_xception(image, xception, verbose: bool = False):
    local_image = torch.permute(image, (1, 2, 0))
    local_image = numpy.expand_dims(local_image, 0)

    return keras.applications.xception.decode_predictions(xception.predict(local_image, verbose=verbose), top=3)[0]

In [ ]:
import torch
import plotly.express as px
import numpy as np
# todo make method
# We look at 8 samples directly
VISUALIZE_SAMPLES: int = 8
image_list: list = []

evaluations: list[tuple] = []

for i in torch.rand(VISUALIZE_SAMPLES):
    image = test[int(i * len(test))][0]
    evaluations.append((evaluate_xception(image, untouched_xception), test[int(i * len(test))][1]))
    image_list.append(torch.permute(image, (1, 2, 0)))

fig = px.imshow(np.array(image_list), binary_string=True, facet_col=0, facet_col_wrap=4)
fig.show()

for i in range(len(evaluations)):
    print(
        f"For facet {i} Xception has prediceted: {[i[1] for i in evaluations[i][0]]} while the true label is {label_mappings[evaluations[i][1]]}")

In [ ]:
# The model does not recognize Muffins as being muffins just as bakery (Which makes sense)
# As the label Muffin is missing in the decoding of the Xception we just map it to bakery

## Model Evaluation without training

In [ ]:
import torch
import keras
# todo make method for all this
predictions: list[tuple[list, int]] = [(evaluate_xception(i[0], untouched_xception), i[1]) for i in test]

In [ ]:
label_mappings = {0: ["chihuahua", "dog"], 1: ["muffin", "bakery"]}
TP = 0 # True positives
for i in range(len(predictions)):
    predicted_values = [j[1].lower() for j in predictions[i][0]]
    true_label = label_mappings[predictions[i][1]]
    
    TP += 1 if set(predicted_values) & set(true_label) else 0
    
precision = TP / len(predictions)
precision # On the top 3 considering the fact that many miss classifications happen for the fact that the labels are more accurate for the dogs (some samples in the training set are not chihuahuas) and that Muffins do not have a real label.

## Fine tuning the model

In [ ]:
from models.structure.augmentation_wrapper import InvertedChannelsAugmentationWrapper


# With augmentation just to permute here
class XceptionAugmented(InvertedChannelsAugmentationWrapper):
    def make_augmentation(self, input_shape: (int, int, int)) -> tuple[keras.Layer, keras.Layer]:
        input_layer = keras.Input(shape=input_shape, name=self.__class__.__name__)
        output_layer = keras.layers.Permute(dims=(2, 3, 1))(input_layer)  # Move channels to be last

        return input_layer, output_layer

    def make_layers(self, input_shape: (int, int, int)) -> tuple[keras.Layer, keras.Layer]:
        inputs = keras.Input(input_shape)
        x = keras.applications.xception.preprocess_input(inputs, data_format="channels_last")
        x = keras.applications.Xception(weights='imagenet', include_top=False)(x, training=False, pooling='max')
        outputs = keras.layers.Dense(1, activation='sigmoid')(x)

        return inputs, outputs

In [ ]:
model = XceptionAugmented().make_model((3, 299, 299))
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.summary()

In [ ]:
history = model.fit(local_train, epochs=20, validation_data=validation_dataloader)

# VGG-16
https://arxiv.org/abs/1409.1556

## Data loading

In [ ]:
import numpy
import numpy as np
from torch.utils.data import DataLoader
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)
test_dataloader = DataLoader(dataset=test, batch_size=16, shuffle=True)

## Model evaulation without training

In [ ]:
untouched_vgg16 = keras.applications.VGG16(weights='imagenet')

In [ ]:
def evaluate_vgg16(image, vgg16, verbose: bool = False):
    local_image = torch.permute(image, (1, 2, 0))
    local_image = numpy.expand_dims(local_image, 0)

    return keras.applications.vgg16.decode_predictions(vgg16.predict(local_image, verbose=verbose), top=3)[0]

In [ ]:
import torch
import plotly.express as px
import numpy as np
# todo make this a method. We repeat it 3 times like idiots
# We look at 8 samples directly
VISUALIZE_SAMPLES: int = 8
image_list: list = []

evaluations: list[tuple] = []

for i in torch.rand(VISUALIZE_SAMPLES):
    image = test[int(i * len(test))][0]
    evaluations.append((evaluate_vgg16(image, untouched_vgg16), test[int(i * len(test))][1]))
    image_list.append(torch.permute(image, (1, 2, 0)))

fig = px.imshow(np.array(image_list), binary_string=True, facet_col=0, facet_col_wrap=4)
fig.show()

for i in range(len(evaluations)):
    print(
        f"For facet {i} VGG16 has prediceted: {[i[1] for i in evaluations[i][0]]} while the true label is {label_mappings[evaluations[i][1]]}")

In [ ]:
import torch
import keras

predictions: list[tuple[list, int]] = [(evaluate_vgg16(i[0], untouched_vgg16), i[1]) for i in test]

In [ ]:
label_mappings = {0: ["chihuahua", "dog"], 1: ["muffin", "bakery"]}
TP = 0 # True positives
for i in range(len(predictions)):
    predicted_values = [j[1].lower() for j in predictions[i][0]]
    true_label = label_mappings[predictions[i][1]]

    TP += 1 if set(predicted_values) & set(true_label) else 0

precision = TP / len(predictions)
precision # On the top 3 considering the fact that many miss classifications happen for the fact that the labels are more accurate for the dogs (some samples in the training set are not chihuahuas) and that Muffins do not have a real label.

## Fine tuning the model

# Resnet-152

## Data loading

## Model evaluation without training

## Fine tuning the model